## Hardware Testbed and Large-scale Testbed Co-simulation

## Co-Simulation

In [1]:
# --- imports ---

import os
import subprocess

import time
import logging
logger = logging.getLogger(__name__)

import csv
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import andes
andes.config_logger(30)
print(andes.__version__)

%matplotlib inline

# --- set logging level ---
# logger.setLevel(logging.WARNING)

1.8.5


In [2]:
is_test = True

rflie = 'datar.csv'
wfile = 'dataw.csv'

if is_test:
    rflie = 't' + rflie

In [3]:
%run -i "core_pre.py"

In [5]:
if is_test:
    sp_ehtb = subprocess.call(f"python3 {path_ehtb} & python3 {path_test}", shell=True)
    # sp_ehtb.wait()

A


Test file in fly
Successfully remove file /Users/jinningwang/Documents/work/hlcosim/ltb/data/tdatar.txt
Emulated data IO start, period = 0.050000
Counter base Updated to 0


### Co-Simulation

In [ ]:
%run -i "core_io.py"

In [ ]:
data_read(file=filer, config=io_config)

In [ ]:
t0_htb = cs_stat['k'] * cs_config['t_step']
tf_htb = t0_htb + cs_config['t_step']

k0 = 0

while cs_stat['k'] <= rows:
    if flag_init:
        # logger.warning("LTB ready, strat HTB to continue.")
        # --- start HTB emulator in test mode ---
        if is_test:
            subprocess.run(f"python3 {path_ehtb}", shell=True)
        while (cs_stat['kr'] != 11):
            # --- repeat reading data until kr==11 ---
            [cs_stat['kr'], cs_stat['p'], cs_stat['q']], txtc, flag_datar = data_read(file=filer, config=io_config)
            kr0 = cs_stat['kr']
            k0 = cs_stat['k']
            # --- reset io_config default values ---
            [io_config['p_df'], io_config['q_df']] = [cs_stat['p'], cs_stat['q']]
        flag_init = False  # Turn off init_flag after first iteration
        cs_stat['k'] = 1
        logger.warning("Co-sim initialized.")
    if cs_stat['kr'] > 10:
        if flag_tc0:
            tc0 = time.time()  # record clock time
            flag_tc0 = False
        # --- data read ---
        # NOTE: repeat reading data until counter update
        iter_read = 0
        k0 = cs_stat['k']
        while (cs_stat['kr'] != kr0 + 1) & (iter_read <= cs_config['itermax_io']):
            [cs_stat['kr'], cs_stat['p'], cs_stat['q']], txtc, flag_datar = data_read(file=filer, config=io_config)
            cs_stat['k'] = cs_stat['ks'] * cs_stat['kr'] + cs_stat['kb']
            [io_config['p_df'], io_config['q_df']] = [cs_stat['p'], cs_stat['q']]
            iter_read += 1
            # update counter base
            if flag_base & (cs_stat['kr'] == 199):
                cs_stat['kr'] = 10
                cs_stat['ks'] += 1
                flag_base = False
                msg = "Counter base updated: %d" % cs_stat['ks']
                logger.warning(msg)
                continue
        tc1 = time.time()  # record clock time
        # --- LTB sim ---
        if (cs_stat['k'] == k0 + 1):
            print('kr, ks, kb, k')
            print(cs_stat['kr'], cs_stat['ks'], cs_stat['kb'], cs_stat['k'])
            if np.mod(cs_stat['k'], 20) == 0:
                msg = "Counter update: k=%d" % cs_stat['k']
                logger.warning(msg)
            # --- send data to HTB ---
            # Make sure `BusFreq` is connected to the load bus
            f_send = ss.BusFreq.get(idx='BusFreq_HTB', src='f', attr='v')  # p.u.
            v_bus = ss.Bus.get(idx=bus_htb, src='v', attr='v')  # RMS, p.u.
            dataw = [v_bus, f_send]  # LTB: voltage, angle
            data_write(dataw=dataw, file=filew, config=io_config)
            tc2 = time.time()  # send end time
            # --- LTB simulation ---
            p_inj = cs_config['load_switch'] * cs_stat['p']
            q_inj = cs_config['load_switch'] * cs_stat['q']
            # a) set PQ data in LTB
            ss.PQ.set(value=p_inj + p0, idx=pq_htb, src='Ppf', attr='v')
            ss.PQ.set(value=q_inj + q0, idx=pq_htb, src='Qpf', attr='v')
            # b) TDS
            ss.TDS.config.tf += cs_config['t_step']
            ss.TDS.run()
            tc3 = time.time()  # record clock time
            # NOTE: tf_htb is the end time of last round
            if tc3 -  tf_htb > cs_config['t_step']:
                cs_stat['iter_fail'] += 1
            cs_stat['tr'] = tc1 - tf_htb  # read time
            cs_stat['tw'] = tc2 - tc1  # write time
            # update HTB time
            t0_htb = cs_stat['k'] * cs_config['t_step']
            tf_htb = t0_htb + cs_config['t_step']
            cs_stat['iter_total'] += 1
            flag_base = True
            #  --- record data ---
            for col in cs_col:
                cs_num[cs_stat['k']-1, cs_col.index(col)] = cs_stat[col]


In [ ]:
filer

In [ ]:
data_read(file=filer, config=io_config)

In [ ]:
cs_stat['k'] == k0 + 1

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 8), dpi=100)
ax[0].scatter(x=range(len(tl)), y=tl)
ax[0].set_ylim([-0.01, 0.06])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 8), dpi=100)
ax[0].scatter(x=range(len(tcl)), y=trl)
ax[0].scatter(x=range(len(tcl)), y=twl)
ax[0].scatter(x=range(len(tcl)), y=tsl)
ax[0].scatter(x=range(len(tcl)), y=tcl)
ax[0].legend(['Read', 'Write', 'Sim', 'Total'])
ax[0].axhline(t_step, color='tab:red')
ax[0].set_xlim([0, len(tcl)])
ax[0].set_title("Data read time interval")
ax[0].set_ylabel("Time [s]")
ax[0].set_ylim([-0.05, 0.12])

ax[1].scatter(x=range(len(crl)), y=crl)
ax[1].set_xlim([0, len(crl)])
ax[1].set_title("Read counter")
ax[1].set_ylabel("Number")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5), dpi=400)

ss.TDS.plt.plot(ss.Bus.v, a=(8),
                ax=ax[0], fig=fig,
                legend=False, show=False,
                title='Load bus voltage',
                ylabel='Voltage [p.u.]')

ss.TDS.plt.plot(ss.GENCLS.omega,
                ax=ax[1], fig=fig,
                legend=False, show=False,
                ytimes=ss.config.freq,
                title='Generator omega',
                ylabel='Frequency [Hz]')

In [ ]:
crl

In [ ]:
plt.plot(range(len(crl)), crl)
plt.title("Counter")
plt.xlabel("Seqence")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5), dpi=400)

# ax[2].scatter(x=np.array(tl), y=np.array(fsl))
ax[0].plot(np.array(tl) - tl[0], np.array(fsl))
# ax[2].set_xlim([0, len(asl)])
ax[0].set_xlim([0, len(crl) * t_step])
ax[0].set_title("Freq. send")
ax[0].set_ylabel("p.u.")
ax[0].set_xlabel("Time [s]")

ax[1].plot(np.array(tl) - tl[0], np.array(vsl))
# ax[2].set_xlim([0, len(asl)])
ax[1].set_xlim([0, len(crl) * t_step])
ax[1].set_title("Volt. send")
ax[1].set_ylabel("p.u.")
ax[1].set_xlabel("Time [s]")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5), dpi=400)

# ax[2].scatter(x=np.array(tl), y=np.array(fsl))
ax[0].plot(np.array(tl) - tl[0], np.array(prl))
# ax[2].set_xlim([0, len(asl)])
ax[0].set_xlim([0, len(crl) * t_step])
ax[0].set_title("P read")
ax[0].set_ylabel("p.u.")
ax[0].set_xlabel("Time [s]")


ax[1].plot(np.array(tl) - tl[0], np.array(qrl))
# ax[2].set_xlim([0, len(asl)])
ax[1].set_xlim([0, len(crl) * t_step])
ax[1].set_title("Q read")
ax[1].set_ylabel("p.u.")
ax[1].set_xlabel("Time [s]")

In [ ]:
ss.dae.ts.y[:, ss.Bus.v.a[8]]

In [ ]:
data1 = pd.DataFrame()
data1['fs'] = fsl
data1['vs'] = vsl
data1['pr'] = prl
data1['qr'] = qrl

data2 = pd.DataFrame()
data2['busv'] = ss.dae.ts.y[:, ss.Bus.v.a[8]]
data2['wg1'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[0]]
data2['wg2'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[1]]
data2['wg3'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[2]]
data2['wg4'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[3]]
data2['wg5'] = ss.dae.ts.x[:, ss.GENCLS.omega.a[4]]

data1.to_csv('data1_2.csv', index=False)
data2.to_csv('data2_2.csv', index=False)